In [1]:
from sagemaker import get_execution_role

sagemaker_role = get_execution_role()

In [2]:
from sagemaker.session import Session

model_data = Session().upload_data(path='model.tar.gz', key_prefix='model')
print('model uploaded to: {}'.format(model_data))

model uploaded to: s3://sagemaker-ap-northeast-1-681108562278/model/model.tar.gz


In [3]:
from sagemaker.tensorflow import TensorFlow
from sagemaker.tensorflow.serving import Model

model = Model(model_data="s3://sagemaker-ap-northeast-1-681108562278/model/model.tar.gz",
              role=sagemaker_role,
              framework_version=1.12
             )

predictor = model.deploy(initial_instance_count=1, instance_type='ml.p2.xlarge')

---------------------------------------------------------------------------------------------------------------!

In [4]:
#import pose_estimate.analysis
import sys
import time
import argparse
import json
import cv2

#from pose_estimate.analysis import predictBatch

import matplotlib
#import pylab as plt
import numpy as np
import util
import os
from numpy import ma
from scipy.ndimage.filters import gaussian_filter

In [5]:
import csv,os

import tensorflow as tf
#from config_reader import config_reader
import scipy
import math

In [6]:
#!pip install opencv-python
#!pip install tensorflow
#!pip install pillow

    100% |████████████████████████████████| 26.6MB 1.9MB/s eta 0:00:01  7% |██▍                             | 2.0MB 56.1MB/s eta 0:00:01
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
ERROR: unknown command "instal" - maybe you meant "install"
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [33]:
cap = cv2.VideoCapture('220_4_full.mp4')

In [34]:
ret, img = cap.read()

In [35]:
imgList = [img]

In [36]:
# partString
partStrList = ["nose", "neck", "Rsho", "Relb", "Rwri", "Lsho", "Lelb", "Lwri", "Rhip", "Rkne", "Rank", "Lhip", "Lkne", "Lank", "Leye", "Reye", "Lear", "Rear", "pt19"]
partStrDict = {x+1:partStrList[x] for x in range(len(partStrList))}
# find connection in the specified sequence, center 29 is in the position 15
limbSeq = [[2,3], [2,6], [3,4], [4,5], [6,7], [7,8], [2,9], [9,10],            [10,11], [2,12], [12,13], [13,14], [2,1], [1,15], [15,17],            [1,16], [16,18], [3,17], [6,18]]
# the middle joints heatmap correpondence
mapIdx = [[31,32], [39,40], [33,34], [35,36], [41,42], [43,44], [19,20], [21,22],           [23,24], [25,26], [27,28], [29,30], [47,48], [49,50], [53,54], [51,52],           [55,56], [37,38], [45,46]]

p_stride=8
p_padValue=128

In [37]:
graph = tf.get_default_graph()

In [36]:
def predictBatch(oriImgList,p_box_size=368, p_scale_search = [0.5,1.,1.5,2.], verbose=False):
    verbose = True
    oriImg = oriImgList[0]
    multiplier = [x * p_box_size / oriImg.shape[0] for x in p_scale_search]

    outList_paf, outList_heatmap = [[] for _ in range(len(oriImgList))],[[] for _ in range(len(oriImgList))]

    for m in range(len(multiplier)):
        scale = multiplier[m]
        input_img_list = []
        for oriImg in oriImgList:
            imageToTest = cv2.resize(oriImg, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)
            imageToTest_padded, pad = util.padRightDownCorner(imageToTest, p_stride, p_padValue)        
            input_img = np.transpose(np.float32(imageToTest_padded[:,:,:,np.newaxis]), (3,0,1,2)) # required shape (1, width, height, channels) 
            input_img_list.append(input_img)
        input_img_list = np.concatenate(input_img_list,axis=0)

        with graph.as_default():
            #out_paf, out_heatmap = model.predict(input_img_list)
            out_paf, out_heatmap = predictor.predict(input_img_list)

        for idx in range(out_heatmap.shape[0]):
            outList_paf[idx].append(out_paf[idx])
            outList_heatmap[idx].append(out_heatmap[idx])

    return outList_heatmap, outList_paf, multiplier, imageToTest_padded, pad

In [38]:
def predictBatch(oriImgList,p_box_size=368, p_scale_search = [0.5,1.,1.5,2.], verbose=False):
    verbose = True
    oriImg = oriImgList[0]
    multiplier = [x * p_box_size / oriImg.shape[0] for x in p_scale_search]

    outList_paf, outList_heatmap = [[] for _ in range(len(oriImgList))],[[] for _ in range(len(oriImgList))]

    for m in range(len(multiplier)):
        scale = multiplier[m]
        input_img_list = []
        for oriImg in oriImgList:
            imageToTest = cv2.resize(oriImg, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)
            imageToTest_padded, pad = util.padRightDownCorner(imageToTest, p_stride, p_padValue)  
            # required shape (1, width, height, channels) 
            input_img = np.transpose(np.float32(imageToTest_padded[:,:,:,np.newaxis]), (3,0,1,2)) 
            input_img_list.append(input_img)
        input_img_list = np.concatenate(input_img_list,axis=0)

        input_dict = {
            'signature_name': 'predict',
            'instances': input_img_list.tolist()
        }
        
        with graph.as_default():
            out = predictor.predict(input_dict)['predictions'][0]
            outList_heatmap, outList_paf = out['heatMat'], out['pafMat']

    return outList_heatmap, outList_paf, multiplier, imageToTest_padded, pad,input_img_list

In [39]:
# 推論を実行
outList_heatmap, outList_paf, multiplier, imageToTest_padded, pad,input_img_list = predictBatch(imgList,p_scale_search=[1.],verbose=False) 

In [22]:
imgList

[None]

In [53]:
payload

'{\n  "0": {\n    "0": [\n      [\n        556.0,\n        158.0\n      ],\n      [\n        496.0,\n        238.0\n      ],\n      [\n        456.0,\n        252.0\n      ],\n      [\n        454.0,\n        382.0\n      ],\n      [\n        496.0,\n        474.0\n      ],\n      [\n        538.0,\n        220.0\n      ],\n      [\n        552.0,\n        322.0\n      ],\n      [\n        572.0,\n        410.0\n      ],\n      [\n        480.0,\n        434.0\n      ],\n      [\n        484.0,\n        568.0\n      ],\n      [\n        478.0,\n        662.0\n      ],\n      [\n        540.0,\n        416.0\n      ],\n      [\n        532.0,\n        552.0\n      ],\n      [\n        522.0,\n        606.0\n      ],\n      [\n        536.0,\n        148.0\n      ],\n      [\n        552.0,\n        142.0\n      ],\n      [\n        490.0,\n        168.0\n      ],\n      [\n        NaN,\n        NaN\n      ]\n    ]\n  },\n  "1": {\n    "0": [\n      [\n        556.0,\n        158.0\n    

In [60]:
enc = json.dumps(input_img_list.tolist())
dec = json.loads(enc)
dec

[[[[0.0, 2.0, 0.0],
   [0.0, 2.0, 0.0],
   [0.0, 2.0, 0.0],
   [0.0, 2.0, 0.0],
   [0.0, 1.0, 0.0],
   [0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0],
   [0.0, 1.0, 0.0],
   [0.0, 1.0, 0.0],
   [0.0, 1.0, 0.0],
   [0.0, 1.0, 0.0],
   [0.0, 1.0, 0.0],
   [0.0, 1.0, 0.0],
   [0.0, 1.0, 0.0],
   [0.0, 1.0, 0.0],
   [0.0, 1.0, 0.0],
   [0.0, 1.0, 0.0],
   [0.0, 1.0, 0.0],
   [0.0, 1.0, 0.0],
   [0.0, 1.0, 0.0],
   [0.0, 1.0, 0.0],
   [0.0, 1.0, 0.0],
   [0.0, 1.0, 0.0],
   [0.0, 1.0, 0.0],
   [0.0, 1.0, 0.0],
   [0.0, 1.0, 0.0],


In [40]:
def predictBatch(oriImgList,p_box_size=368, p_scale_search = [0.5,1.,1.5,2.], verbose=False):
    verbose = True
    oriImg = oriImgList[0]
    multiplier = [x * p_box_size / oriImg.shape[0] for x in p_scale_search]

    outList_paf, outList_heatmap = [[] for _ in range(len(oriImgList))],[[] for _ in range(len(oriImgList))]

    for m in range(len(multiplier)):
        scale = multiplier[m]
        input_img_list = []
        for oriImg in oriImgList:
            imageToTest = cv2.resize(oriImg, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)
            imageToTest_padded, pad = util.padRightDownCorner(imageToTest, p_stride, p_padValue)  
            # required shape (1, width, height, channels) 
            input_img = np.transpose(np.float32(imageToTest_padded[:,:,:,np.newaxis]), (3,0,1,2)) 
            input_img_list.append(input_img)
        input_img_list = np.concatenate(input_img_list,axis=0)

    return input_img_list

In [41]:
input_img_list = predictBatch(imgList,p_scale_search=[1.],verbose=False)

In [47]:
import boto3
import json

client = boto3.client('sagemaker-runtime')
    
input_dict = {
    'signature_name': 'predict',
    'instances': input_img_list.tolist(),
}

with graph.as_default():
    response = client.invoke_endpoint(
        EndpointName='sagemaker-tensorflow-serving-2019-05-22-23-29-27-399',
        ContentType='application/json',
        Body=json.dumps(input_dict)
    )
    result_body = json.load(response['Body'])
    print(result_body['predictions'][0])
    

[[[0.001028, 0.00122445, 0.00101969, 0.000825006, 0.00076298, 0.00109966, 0.000987137, 0.000928411, 0.000783417, 0.0004933, 0.000508194, 0.000891756, 0.000541709, 0.000362776, 0.00105483, 0.000796751, 0.00094663, 0.000757655, 0.993631], [0.000774853, 0.000691905, 0.000696156, 0.000483537, 0.000460319, 0.000741898, 0.000565565, 0.000579889, 0.000632779, 0.000378333, 0.000414979, 0.00061387, 0.000347205, 0.000280019, 0.000721616, 0.000619419, 0.00062899, 0.000499903, 0.995003], [0.000497082, 0.000347795, 0.000455957, 0.000294507, 0.000269604, 0.000504727, 0.000347204, 0.000378409, 0.000515954, 0.000291185, 0.000354525, 0.000446762, 0.000238408, 0.0002279, 0.000408688, 0.000422915, 0.000363685, 0.000279546, 0.996493], [0.00072849, 0.000433874, 0.000506139, 0.00024783, 0.000242273, 0.000532076, 0.000292663, 0.000344552, 0.00044252, 0.000222083, 0.000320597, 0.00035902, 0.000160984, 0.000193775, 0.000591805, 0.000517464, 0.000568323, 0.000442219, 0.996429], [0.000628436, 0.000335906, 0.0004

In [51]:
import boto3
import json

client = boto3.client('sagemaker-runtime')

with open("movie.json", "r") as f:
    payload = f.read()

response = client.invoke_endpoint(
    EndpointName='sagemaker-tensorflow-serving-2019-05-20-06-13-36-245',
    ContentType='application/json',
    #Body=json.dumps(input_img_list.tolist()),
    Body='{"signature_name": "predict", "instances":json.dumps(input_img_list.tolist()) }'
)

result_body = json.load(response['Body'])
print(result_body)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from model with message "{ "error": "JSON Parse error: Invalid value. at offset: 42" }". See https://ap-northeast-1.console.aws.amazon.com/cloudwatch/home?region=ap-northeast-1#logEventViewer:group=/aws/sagemaker/Endpoints/sagemaker-tensorflow-serving-2019-05-20-06-13-36-245 in account 681108562278 for more information.

# sample

In [1]:
from sagemaker.session import Session

model_data = Session().upload_data(path='mobilenet.tar.gz', key_prefix='model')
print('model uploaded to: {}'.format(model_data))

model uploaded to: s3://sagemaker-ap-northeast-1-681108562278/model/mobilenet.tar.gz


In [3]:
from sagemaker import get_execution_role

sagemaker_role = get_execution_role()

In [4]:
from sagemaker.tensorflow.serving import Model

# Use an env argument to set the name of the default model.
# This is optional, but recommended when you deploy multiple models
# so that requests that don't include a model name are sent to a 
# predictable model.
env = {'SAGEMAKER_TFS_DEFAULT_MODEL_NAME': 'mobilenet_v2_140_224'}

model = Model(model_data=model_data, role=sagemaker_role, framework_version=1.12, env=env)
predictor = model.deploy(initial_instance_count=1, instance_type='ml.c5.xlarge')

----------------------------------------------------------------------------!